# Part 2
On the assignment link you’ll find an sqlite database called northwind.sqlite. This database originally comes from https://github.com/jpwhite3/northwind-SQLite3 but has been modified. Imaging that you have been hired by this firm to help them summarize and understand how good their employees are in terms of their sales. You’ll write up a report for the company (so they are your audience when writing). Your report will be in the form of an .html notebook.

You should drill down into things like how they’ve done across years, how they do in different regions, etc. Please make sure there is a narrative throughout the report. The actual way you quantify how the employees are doing is up to you to determine (not every group will investigate the same things). Please note that the employees have different roles so not everyone is entirely focused on sales. You’ll want to study the database tables a bit to get an idea about the things you could investigate:

You should provide summary statistics and graphs with corresponding interpretations for the company - no need to do any modeling, just focus on summarizing employee performance in meaningful ways. You’ll need to combine data. You can do the joins, summaries, etc. through pandas or via SQL. There are a couple of tables that are empty, just ignore those. Make sure your report has clear sections for the company representatives to read!

In [1]:
#libraries
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
con = sqlite3.connect("northwind.sqlite")

In [3]:
pd.read_sql("SELECT * FROM sqlite_schema WHERE type = 'table';", con)

,type,name,tbl_name,rootpage,sql
0,table,Categories,Categories,2,CREATE TABLE [Categories]\n( [CategoryID]...
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,CustomerCustomerDemo,CustomerCustomerDemo,28,CREATE TABLE [CustomerCustomerDemo](\n [Cust...
3,table,CustomerDemographics,CustomerDemographics,30,CREATE TABLE [CustomerDemographics](\n [Cust...
4,table,Customers,Customers,32,CREATE TABLE [Customers]\n( [CustomerID] ...
5,table,Employees,Employees,38,CREATE TABLE [Employees]\n( [EmployeeID] ...
6,table,EmployeeTerritories,EmployeeTerritories,68,CREATE TABLE [EmployeeTerritories](\n [Emplo...
7,table,Order Details,Order Details,70,CREATE TABLE [Order Details](\n [OrderID]INT...
8,table,Orders,Orders,92,CREATE TABLE [Orders](\n [OrderID]INTEGER NO...
9,table,Products,Products,128,CREATE TABLE [Products](\n [ProductID]INTEGE...


We are first going to create a large dataframe that will contains all of the variables that we are interested in using to assess employee sales. We will do this in a series of steps, ultimately sequentially merging three dataframes named df1, df2, and df3.

We will first combine the employee details and the Territories from table `Employees` and `EmployeeTerritories`. We will only look at the sales representatives.

In [6]:
#This dataframe is Employee ID, name, region and territory
df1 = pd.merge(left = pd.read_sql("SELECT  EmployeeID, Title, LastName, FirstName, Region FROM Employees WHERE Title = 'Sales Representative';", con),
        right = pd.read_sql("SELECT  EmployeeID, TerritoryID FROM EmployeeTerritories;", con),
        how = "inner",
        on = "EmployeeID")
#df1.head()

In [7]:
#df2 adds orderID, CustomerID EmployeeID and OrderDate
df2 = pd.merge(left = df1,
              right = pd.read_sql("SELECT OrderID, CustomerID, EmployeeID, OrderDate FROM Orders", con),
              how = "inner",
              on = "EmployeeID")
#df2.head(10)

In [8]:
#df3 adds all the columns from the table 'Order Details'
df3 = pd.merge(left = df2,
               right = pd.read_sql("SELECT * FROM 'Order Details'", con),
               how = "inner",
               on = "OrderID")
# chage the date to year
df3['OrderYear'] = [x[:4] for x in df3.OrderDate]
df3 = df3.drop('OrderDate', axis = 1)
#df3.head(10)

In [ ]:
# calculate the amount for one product within one order
df3['Total'] = df3['UnitPrice']*df3['Quantity']*(1-df3['Discount'])
#df3.head(10)

# Part 3
For part 3, we’re going to redo some of the homework 6 NFL data analysis via spark. (This part should be very easy/short! We’ll use spark SQL and/or pandas-on-spark functionality rather than writing our own MapReduce type code.) 

To do:
• Read in the full nfl data set into spark as a spark data frame
• Use spark SQL to find the mean and standard deviation for the AQ1, AQ2, AQ3, AQ4, AQFinal, HQ1, HQ2, HQ3, HQ4, and HFinal variables
• Find the mean and standardeviation for those same columns but do so for each value of the season variable

Now, repeat the three steps above but use pandas-on-spark rather than spark SQL! As always, you should have a basic narrative flowing through what you are doing and an interpretation of any stats/graphs created.